In [1]:
import collections
import networkx as nx
import sys
import os
sys.path.append(r'c:\\Users\\rotim\\OneDrive\\Documents\\Reading\\graph-code\\Skoltech-PhD-Thesis\\_codes\\SIR\\Influential-nodes-identification-in-complex-networks-via-information-entropy')
from algorithms import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle

In [2]:
graph_collections, graph_name = read_graph("./test-graphs")


{'ABILENE': {'index': 0, 'nodes': 11, 'edges': 14}, 'ABVT': {'index': 1, 'nodes': 23, 'edges': 31}, 'AGIS': {'index': 2, 'nodes': 25, 'edges': 30}, 'ARN.GRAPHML': {'index': 3, 'nodes': 30, 'edges': 29}, 'BELLCANADA': {'index': 4, 'nodes': 48, 'edges': 64}, 'BELLSOUTH.GRAPHML': {'index': 5, 'nodes': 51, 'edges': 66}, 'BICS.GRAPHML': {'index': 6, 'nodes': 33, 'edges': 48}, 'CERNET.GRAPHML': {'index': 7, 'nodes': 41, 'edges': 58}, 'CESNET200706.GRAPHML': {'index': 8, 'nodes': 44, 'edges': 51}, 'CHINANET.GRAPHML': {'index': 9, 'nodes': 42, 'edges': 66}, 'COGENTCO.GRAPHML': {'index': 10, 'nodes': 197, 'edges': 243}, 'COLT.GRAPHML': {'index': 11, 'nodes': 153, 'edges': 177}, 'DELTACOM.GRAPHML': {'index': 12, 'nodes': 113, 'edges': 161}, 'GARR201110.GRAPHML': {'index': 13, 'nodes': 59, 'edges': 74}, 'GEANT2001': {'index': 14, 'nodes': 27, 'edges': 38}, 'GRNET.GRAPHML': {'index': 15, 'nodes': 37, 'edges': 42}, 'INTELLIFIBER.GRAPHML': {'index': 16, 'nodes': 73, 'edges': 95}, 'INTEROUTE.GRAPHML'

In [3]:
# India   145nodes 187edges #37
# US carrier 158nodes   189edges # 41
# Kentucky    754nodes   895edges # 20
# Albeni  # 0
# Geant  2001 #14
# Bell Canada # 4
# Abvt # 1
# Agis # 2
# 10, 18, 8
selection__= [37,41,20,0,14,4,1,2,10, 18, 8]

In [4]:
G = graph_collections[37]
# geo_loc_data = get_geo_data("Internet2LatLong.csv")
geo_loc_data = {k: {"lat": v.get('Latitude', 0), "long": v.get('Longitude', 0)}
                for (k, v) in G.nodes(data=True)}
# print(geo_loc_data)
edge_geo_data_combined = assign_location(G, geo_loc_data)
# set edge attributes
set_edge_attr(G, edge_geo_data_combined)


In [5]:
gGrahps = [(j, graph_collections[k]) for i, j in enumerate(graph_name)
 for k in selection__ if i == k]


geo_loc_data_sample = {i: {k: {"lat": v.get('Latitude', 0), "long": v.get('Longitude', 0)}
     for (k, v) in g.nodes(data=True)} for i, g in gGrahps}
# print(geo_loc_data_sample)
edge_geo_data_sample = {k: assign_location(u, v) for (k, v), (j,u) in zip(geo_loc_data_sample.items(), gGrahps)}

[set_edge_attr(g[1], v[1]) for (v ,g) in zip(edge_geo_data_sample.items(), gGrahps)]


# # edge_geo_data_combined_sample = {k: assign_location(g, v) for (k, v), g in zip(geo_loc_data_all.items(), graph_collections)}


[None, None, None, None, None, None, None, None, None, None, None]

In [6]:
weight_sum =sum([sum(hub_information(G, i, 2))
     for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))])
node_strength_2_deg = [(i, sum(hub_information(G, i, 2)))  # /weight_sum
                       for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))]
degree__ = [(i, len(n_neighbor(G, i, 1))) for (i, j) in sorted(list(G.degree()),
                                                               key=lambda item: int(item[0]))]


In [7]:

source_1, source_2, source_3, source_4, *others = degree__, node_strength_2_deg, eccentricity(G),  approximate_current_flow_betweenness_centrality(G, weight="weight"), current_flow_betweenness_centrality(G), betweenness_centrality(G, weight="weight"),  closeness_centrality(G),      load_centrality(
    G),    #eigenvector_centrality(G),

# print(source_2, source_3)


In [8]:
sources_of_evidences = {
    "Degree": {g[0]: [(i, len(n_neighbor(g[1], i, 1))) for (i, j) in sorted(list(g[1].degree()), key=lambda item: int(item[0]))] for g in gGrahps},
    "Node Strength": {g[0]: [(i, sum(hub_information(g[1], i, 2))) for (i, j) in sorted(list(g[1].degree()), key=lambda item: int(item[0]))] for g in gGrahps},
    "Eccentricity": {g[0]: eccentricity(g[1]) for g in gGrahps},
    "Approx Curr. Flow Betwns Cent'y": {g[0]: approximate_current_flow_betweenness_centrality(g[1]) for g in gGrahps}
}
# sources_of_evidences["Degree"]["ABILENE"]

In [9]:
e_v = 2
varying_examples_multi(source_1, source_2, source_3, source_4,e_v)

[('70', 0.9986942454915201),
 ('52', 0.7269206963021477),
 ('118', 0.6606933873804312),
 ('37', 0.04551598153655445)]

In [10]:
evis = [2,3,4]
selected_nodes_based_on_source ={i:{e: varying_examples_multi(sources_of_evidences["Degree"][i],
sources_of_evidences["Node Strength"][i], 
sources_of_evidences["Eccentricity"][i], 
    sources_of_evidences["Approx Curr. Flow Betwns Cent'y"][i], e) for e in evis} for i, j in gGrahps}

selected_nodes_based_on_source_ext = {k2:{k:len(v) for k,v in v2.items()} for k2,v2 in selected_nodes_based_on_source.items()}

# {i: { "#Nodes":  nx.number_of_nodes(j), "#Edges": nx.number_of_edges(j), "sel_":v} for (i,j), v in zip(gGrahps, selected_nodes_based_on_source.values())}
sel_node_summary={i: {"#Nodes":  nx.number_of_nodes(j), "#Edges": nx.number_of_edges(j), "Diameter":nx.diameter(j), "Density":nx.density(j)}for (i, j)in gGrahps }
sel_to_table = {
    i: (sel_node_summary[i] | selected_nodes_based_on_source_ext[i]) for i, j in gGrahps}
pd.DataFrame.from_dict(
    sel_to_table, orient='index')
print()


In [11]:
selected_nodes_based_on_source


{'ABILENE': {2: [('7', 0.999028599121206),
   ('8', 0.9931467209871861),
   ('6', 0.7571798128170234),
   ('9', 0.7562986118582249),
   ('5', 0.7016084028819796),
   ('4', 0.5915030963200972),
   ('10', 0.5603575276295959)],
  3: [('7', 20.63709230769231),
   ('9', 4.899784087548063),
   ('10', 4.518380874484546),
   ('8', 2.2816482150926345),
   ('1', 0.04651503059323758)],
  4: [('7', 45.61117734978671),
   ('5', 43.72912939894377),
   ('3', 29.9717959961976),
   ('2', 22.347721362374543),
   ('6', 11.564678498341685),
   ('1', 7.691552120371613)]},
 'ABVT': {2: [('11', 0.9991533889283354),
   ('1', 0.8102800440940692),
   ('10', 0.43433003311276563),
   ('5', 0.32810873941933705)],
  3: [('1', 17.705885382059822),
   ('21', 5.1456154669497876),
   ('20', 4.445703701564566),
   ('9', 4.175141405691684),
   ('7', 4.155361677288438),
   ('19', 4.116508972319038),
   ('14', 4.0146041607935565),
   ('16', 3.941490836598527),
   ('15', 3.501589414817144),
   ('22', 2.638269762929372),
   

In [23]:
selected_nodes_based_on_source["ABILENE"]
# {i:{varying_examples_multi(j,v2) for i,j in gGrahps for v2 in v} for v in selected_nodes_based_on_source.values()}
# varying_examples_multi(j)
# cluster_optimal_nodes()
# selected_nodes_based_on_source["BELLCANADA"][4]
# sel_error ={k2: {k: cluster_optimal_nodes(j, v) for k, v in v2.items() for i, j in gGrahps if {i, k2} } for k2, v2 in selected_nodes_based_on_source.items()}
clusters__ = {k2: {k: cluster_optimal_nodes(j, v,2) for k, v in v2.items()}  for (i, j), (k2, v2) in zip(gGrahps, selected_nodes_based_on_source.items())}



In [30]:
# gGrahps[0][1]
cluster_optimal_nodes(gGrahps[0][1], selected_nodes_based_on_source["ABILENE"][2])


[('7', set()),
 ('8', set()),
 ('6', {'3'}),
 ('9', {'0', '2'}),
 ('5', set()),
 ('4', set()),
 ('10', {'1'})]

In [37]:
def cluster_optimal_nodes_test(G, opti_rank, b=1):
    node_list = G.nodes()
    opti_rank_nodes = [i for i, j in opti_rank]
    remainder_nodes = set(node_list) - set(opti_rank_nodes)
    current_set_result_all = []

    while remainder_nodes != set():
        chosen_list = [(i, n_neighbor(G, i, b)) for (i, j) in opti_rank]
        for x, y in chosen_list:
            chosen_set = set(y)
            current_set = chosen_set.intersection(remainder_nodes)
            current_set_result = (x, current_set)
            current_set_result_all.append((current_set_result))
            remainder_nodes -= chosen_set
        b += 1
    coll_ = collections.defaultdict(list)
    for d, e in current_set_result_all:
        coll_[d].extend(e)  # add to existing list or create a new one

    current_set_result_all = list(coll_.items())
    print(current_set_result_all)

    return [(i, set(j)) for i, j in current_set_result_all]


In [38]:
cluster_optimal_nodes_test(gGrahps[0][1], selected_nodes_based_on_source["ABILENE"][2])


[('7', []), ('8', []), ('6', ['3']), ('9', ['2', '0']), ('5', []), ('4', []), ('10', ['1'])]


[('7', set()),
 ('8', set()),
 ('6', {'3'}),
 ('9', {'0', '2'}),
 ('5', set()),
 ('4', set()),
 ('10', {'1'})]